In [1]:
#from __future__ import print_function
import argparse
import os
import imp
import algorithms as alg
from dataloader import DataLoader, GenericDataset

In [2]:
import torch
import torch.nn as nn
torch.cuda.device_count()

1L

In [3]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [4]:
args_opt = Namespace(
    exp = 'Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features-FineTune',
    evaluate = False,
    checkpoint = 12,
    num_workers = 1,
    cuda = True,
    disp_step= 5
)

In [5]:
exp_config_file = os.path.join('.','config',args_opt.exp+'.py')
# if args_opt.semi == -1:
exp_directory = os.path.join('.','experiments',args_opt.exp)
# else:
#    assert(args_opt.semi>0)
#    exp_directory = os.path.join('.','experiments/unsupervised',args_opt.exp+'_semi'+str(args_opt.semi))

In [6]:
exp_config_file

'./config/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features-FineTune.py'

In [7]:
# Load the configuration params of the experiment
print('Launching experiment: %s' % exp_config_file)
config = imp.load_source("",exp_config_file).config
config['exp_dir'] = exp_directory # the place where logs, models, and other stuff will be stored
print("Loading experiment %s from file: %s" % (args_opt.exp, exp_config_file))
print("Generated logs, snapshots, and model files will be stored on %s" % (config['exp_dir']))

Launching experiment: ./config/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features-FineTune.py
Loading experiment Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features-FineTune from file: ./config/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features-FineTune.py
Generated logs, snapshots, and model files will be stored on ./experiments/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features-FineTune


In [8]:
# Set train and test datasets and the corresponding data loaders
data_train_opt = config['data_train_opt']
data_test_opt = config['data_test_opt']
num_imgs_per_cat = data_train_opt['num_imgs_per_cat'] if ('num_imgs_per_cat' in data_train_opt) else None

In [9]:
dataset_train = GenericDataset(
    dataset_name=data_train_opt['dataset_name'],
    split=data_train_opt['split'],
    random_sized_crop=data_train_opt['random_sized_crop'],
num_imgs_per_cat=num_imgs_per_cat)

custom


In [10]:
dataset_test = GenericDataset(
    dataset_name=data_test_opt['dataset_name'],
    split=data_test_opt['split'],
    random_sized_crop=data_test_opt['random_sized_crop'])

custom


In [11]:
dloader_train = DataLoader(
    dataset=dataset_train,
    batch_size=data_train_opt['batch_size'],
    unsupervised=data_train_opt['unsupervised'],
    epoch_size=data_train_opt['epoch_size'],
    num_workers=args_opt.num_workers,
shuffle=True)

In [12]:
dloader_test = DataLoader(
    dataset=dataset_test,
    batch_size=data_test_opt['batch_size'],
    unsupervised=data_test_opt['unsupervised'],
    epoch_size=data_test_opt['epoch_size'],
    num_workers=args_opt.num_workers,
shuffle=False)

In [13]:
len(dloader_train.dataset)

64000

In [14]:
config['disp_step'] = args_opt.disp_step

In [15]:
algorithm = getattr(alg, config['algorithm_type'])(config)

2019-05-13 13:18:05,993 - algorithms.Algorithm - INFO   - Algorithm options {'max_num_epochs': 40, 'disp_step': 5, 'algorithm_type': 'FeatureClassificationModel', 'criterions': {'loss': {'opt': None, 'ctype': 'CrossEntropyLoss'}}, 'data_test_opt': {'unsupervised': False, 'batch_size': 150, 'epoch_size': None, 'random_sized_crop': False, 'split': 'supervised/val', 'dataset_name': 'custom'}, 'exp_dir': './experiments/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features-FineTune', 'out_feat_keys': ['conv5'], 'networks': {'classifier': {'opt': [{'cls_type': 'Alexnet_conv5', 'num_classes': 1000, 'nChannels': 256}], 'def_file': 'architectures/MultipleNonLinearClassifiers.py', 'optim_params': {'LUT_lr': [(10, 0.0001), (20, 5e-05), (30, 0.0004), (40, 8e-05)], 'nesterov': True, 'optim_type': 'sgd', 'lr': 0.001, 'weight_decay': 0.0005, 'momentum': 0.9}, 'pretrained': './experiments/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features/models/classifier_net_epoch6'}, 'feat_extracto

('Number of classifiers: ', 1)


2019-05-13 13:18:10,007 - algorithms.Algorithm - INFO   - ==> Load pretrained parameters from file ./experiments/Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features/models/classifier_net_epoch6:
2019-05-13 13:18:42,333 - algorithms.Algorithm - INFO   - Set network feat_extractor
2019-05-13 13:18:42,968 - algorithms.Algorithm - INFO   - ==> Initiliaze network feat_extractor from file architectures/AlexNet.py with opts: {'num_classes': 4}
2019-05-13 13:18:44,110 - algorithms.Algorithm - INFO   - ==> Load pretrained parameters from file ./experiments/Custom_RotNet_AlexNet/models/model_net_epoch13:
2019-05-13 13:18:45,736 - algorithms.Algorithm - INFO   - Initialize criterion[loss]: CrossEntropyLoss with options: None


In [16]:
# device = torch.device("cuda")
# device

In [17]:
if args_opt.cuda: # enable cuda
#     if torch.cuda.device_count() > 1:
#         algorithm = nn.DataParallel(algorithm)
    algorithm.load_to_gpu()
#     algorithm.to(device)
if args_opt.checkpoint > 0: # load checkpoint
    algorithm.load_checkpoint(args_opt.checkpoint, train= (not args_opt.evaluate))

2019-05-13 13:18:46,352 - algorithms.Algorithm - INFO   - Load checkpoint of epoch 12
2019-05-13 13:18:49,883 - algorithms.Algorithm - INFO   - Initialize optimizer: sgd with params: {'LUT_lr': [(10, 0.0001), (20, 5e-05), (30, 0.0004), (40, 8e-05)], 'nesterov': True, 'optim_type': 'sgd', 'lr': 0.001, 'weight_decay': 0.0005, 'momentum': 0.9} for netwotk: classifier
2019-05-13 13:18:50,276 - algorithms.Algorithm - INFO   - Initialize optimizer: adam with params: {'optim_type': 'adam', 'lr': 0.001, 'beta': (0.9, 0.999), 'LUT_lr': [(10, 0.0001), (20, 5e-05), (30, 0.0004), (40, 8e-05)]} for netwotk: feat_extractor


In [ ]:
if not args_opt.evaluate: # train the algorithm
    algorithm.solve(dloader_train, dloader_test)
else:
    algorithm.evaluate(dloader_test) # evaluate the algorithm


2019-05-13 13:18:53,769 - algorithms.Algorithm - INFO   - Training epoch [ 13 /  40]
2019-05-13 13:18:53,771 - algorithms.Algorithm - INFO   - ==> Set to classifier optimizer lr = 0.0000500000
2019-05-13 13:18:53,774 - algorithms.Algorithm - INFO   - ==> Set to feat_extractor optimizer lr = 0.0000500000
2019-05-13 13:18:53,776 - algorithms.Algorithm - INFO   - Training: Custom_NonLinearClassifiers_ImageNet_RotNet_AlexNet_Features-FineTune
  0%|          | 0/427 [00:00<?, ?it/s]

torch.Size([150, 256, 13, 13])


  0%|          | 2/427 [00:21<1:48:10, 15.27s/it]

torch.Size([150, 256, 13, 13])
torch.Size([150, 256, 13, 13])


  1%|          | 3/427 [00:26<1:24:51, 12.01s/it]

...

 11%|█         | 47/427 [06:54<51:18,  8.10s/it]

torch.Size([150, 256, 13, 13])


 11%|█         | 48/427 [07:00<46:32,  7.37s/it]

torch.Size([150, 256, 13, 13])


 11%|█▏        | 49/427 [07:11<54:17,  8.62s/it]

/scratch/um367/urwa-env/py2.7.12/lib/python2.7/site-packages/torchvision/transforms/functional.py", line 48, in to_tensor